In [1]:
import sys
sys.path.append(r"/home/kayzhou/zhangyue/text/text_UNION/test_GCN_LSTM")
from model import GraphConvolution, GCNModel
from utils import *
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pickle as pkl
import os
import sys
import time
import argparse
from sklearn.metrics import f1_score
#os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
class Trainer:
    def __init__(self, args):

        print(args)
        
        # Define Parameters
        self.DATASET = args['dataset']
        self.NB_EPOCH = args['epochs']
        self.LR = args['learnrate']
        self.L2 = args['l2norm']
        self.HIDDEN = args['hidden']
        self.BASES = args['bases']
        self.use_cuda = True #原本为True
        self.TEXT_MODEL = args['text_model']
        self.SUPERVISE_FLAG = args['supervise_flag']   
        self.PRED_TYPE = args['pred_type']   
        self.USE_BIAS = True
        self.MODEL_NAME = '%s_%s_%s' % (self.DATASET, self.TEXT_MODEL, self.SUPERVISE_FLAG) # '_64hlstm30_withtext'
        self.num_features = 16
        self.log_step = 1
        

        # Load Data
        #去掉文件名返回目录
        dirname = "/home/kayzhou/zhangyue/text/data_GCN/data_processed/"
        #utils load_data
        raw_data, data = load_data(
                args, dirname, self.use_cuda, self.SUPERVISE_FLAG)
        #A = raw_data['A']
        y = raw_data['y']
        #Series.to_dense()函数已返回给定系列对象的密集表示形式。
        #它分配了内存以存储系列中的缺失值。当缺少大量数据时，密集表示对内存的效率不高。
        y = np.array(y)
        self.labels = y

        self.idx_train = data['idx_train'] 
        self.idx_valid = data['idx_valid']  
        self.idx_test= data['idx_test'] 
        self.idx_train_set = data['idx_train_set'] 
        self.idx_test_set = data['idx_test_set'] 
        self.inputs = data['inputs']
        self.labels_train = data['labels_train'] 
        self.labels_valid = data['labels_valid']  
        self.labels_test = data['labels_test']  
        self.num_nodes = data['num_nodes']  
        self.num_docs = data['num_docs']  
        self.num_non_docs= data['num_non_docs']
        self.node2adj = data['node2adj']
        self.support = data['support']
        self.idx_train_pre = (np.array(self.idx_train)+self.num_non_docs).tolist()
        self.idx_valid_pre = (np.array(self.idx_valid)+self.num_non_docs).tolist()
        self.idx_test_pre = (np.array(self.idx_test)+self.num_non_docs).tolist()
        self.text_batch_size = 1000
     


        print('Data loaded successfully!')

        # Compile Model
        #定义模型的基本参数
        
        self.model = GCNModel(data, self.num_features, self.num_nodes, self.HIDDEN, self.support, self.BASES, 2, 
            text_model=self.TEXT_MODEL, bias_feature=self.USE_BIAS)
        if (self.use_cuda):
            self.model = self.model.cuda()
            #self.model = self.model

        parameters = [p for p in list(self.model.parameters())
            if p.requires_grad]
        # for p in parameters:
        #     print(p.size())
        self.optimizer = optim.Adam(parameters,
                            lr=self.LR, weight_decay=self.L2)

        #lstm为sum
        self.cross_entropy_loss = nn.CrossEntropyLoss(reduction='sum')

    
    def _train(self):
        # Fit Model
        best_acc, best_test_acc = 0, 0
        best_loss = 10000000
        best_loss_train = 1000000
        best_epoch = 0
        
        loss_list = []
        train_acc_list = []
        dev_acc_list = []
        test_acc_list = []
        
        for epoch in range(1, self.NB_EPOCH + 1):
            # Log wall-clock time
            t = time.time()

            self.model.zero_grad()

            # Single training iteration
            embeds_0 = self.inputs[0] 
            embeds_1 = self.model.gc1([embeds_0] + self.inputs[1:])
            embeds_2 = self.model.gc2([embeds_1] + self.inputs[1:])
            embeds_final = embeds_2
            scores = self.model.clf_bias(embeds_2)
            loss_train = self.cross_entropy_loss(scores[self.idx_train_pre], self.labels_train) 

            #调整loss的系数
            #loss_train *= 10
            loss_train.backward()
            self.optimizer.step()
            
            #猜测doc节点的编号是在最后，所以利用以下公式
            #np.random.permutation对range()之间的序列进行随机排序
            ##self.num_non_docs=num_nodes - num_docs
            text_idx_perm = np.random.permutation(range(self.num_non_docs, self.num_nodes))
            #如果预测类型是text或者all则运行
            #如果TEXT_MODEL不是HLSTM且预测类型是text或者all只循环一次

            for start in range(0, self.num_docs, self.text_batch_size):
                if (self.PRED_TYPE not in ['text', 'all']):
                    break
                self.model.zero_grad()
                end = start + self.text_batch_size
                if (end > self.num_docs):
                    end = self.num_docs
                doc_idx_list_raw = text_idx_perm[start:end]
                #inputs 在utils可找到
                embeds_0 = self.inputs[0] # model.input_layer()
                #[1，2]+[2，3]=[1,2,2,3]
                embeds_1 = self.model.gc1([embeds_0] + self.inputs[1:])
                embeds_2 = self.model.gc2([embeds_1] + self.inputs[1:])
                embeds_final = embeds_2
                
                doctext_idx_list, docnode_pos_idx, docnode_neg_idx, doctext_flag = [], [], [], []
                for tidx, doctext_idx in enumerate(doc_idx_list_raw):
                    doctext_idx_list.append(doctext_idx)
                    
                    docnode_pos_idx.append(doctext_idx) 
                    #pos集合增加1个neg集合增加5个
                    docnode_neg_idx.append(np.random.randint(self.num_non_docs, self.num_nodes, size=(5)))
                    
                    if (doctext_idx in self.idx_train_set):
                        doctext_flag.append(tidx)
                        
                docnode_pos_idx = np.array(docnode_pos_idx).reshape(-1, 1)
                docnode_neg_idx = np.array(docnode_neg_idx)
                docnode_idx_list = np.concatenate((docnode_pos_idx, docnode_neg_idx), axis=1)

                #doctext_idx_list = torch.LongTensor(doctext_idx_list)
                doctext_idx_list = torch.LongTensor(doctext_idx_list).cuda()
                batch_input = self.model.input_layer.get_doc_embed(doctext_idx_list-self.num_non_docs)
                docnode_idx_list = torch.LongTensor(docnode_idx_list).cuda()
                #docnode_idx_list = torch.cuda.LongTensor(docnode_idx_list)
                #docnode_idx_list = torch.LongTensor(docnode_idx_list)
                #取出docnode索引所对应的embeds_final的维度1，再将其维度进行重构
                batch_target = embeds_final.index_select(0, docnode_idx_list.view(-1)).view(-1, 6, self.num_features)
                
                #-1表示自定计算改维度数值
                #batch_input表示doc节点嵌入
                #torch.bmm矩阵乘法
                batch_scores = torch.bmm(batch_target, 
                    batch_input.view(-1, self.num_features, 1)).squeeze()
                batch_labels = torch.LongTensor([0] * len(doctext_idx_list)).cuda()
                #batch_labels均设置为0
                #batch_labels = torch.LongTensor([0] * len(doctext_idx_list))
                loss_text_node = self.cross_entropy_loss(batch_scores, batch_labels)

                if (len(doctext_flag) > 0):
                    #self.model.clf_bias为一个线性层
                    scores_text = self.model.clf_bias(batch_input)
                    loss_text = self.cross_entropy_loss(scores_text[doctext_flag], 
                                                        torch.LongTensor(self.labels[docnode_pos_idx-self.num_non_docs][doctext_flag]).cuda().view(-1))
                    # loss_text = self.cross_entropy_loss(scores_text[doctext_flag], 
                    #                                     torch.LongTensor(self.labels[docnode_pos_idx-self.num_non_docs][doctext_flag]).view(-1))
                            #torch.LongTensor(self.labels[docnode_pos_idx][doctext_flag])..view(-1))
                            
                    
                else: 
                    loss_text = torch.tensor(0).cuda()
                    #loss_text = torch.tensor(0)
                #HLSTM模型一个epoch比别的text模型多反向传播
                loss_train = loss_text_node + loss_text
                loss_train.backward()
                self.optimizer.step()



            #上一个if结束


            loss_list.append(loss_train)
            print(str(epoch)+"--loss_train")
            print(loss_train)
            # Evaluate model and save the best one based on performance on validataion set
            if epoch % self.log_step == 0:

                preds_test,labels_test,loss_train_val, loss_valid_val, loss_test_val,train_acc_sel, valid_acc_sel, test_acc_sel, preds_sel, result_table = self._evaluate()
                
                train_acc_list.append(train_acc_sel)
                dev_acc_list.append(valid_acc_sel)
                test_acc_list.append(test_acc_sel)

                f1 = f1_score(preds_test.cpu(), labels_test.cpu(),average="macro")
                print("Epoch: {:04d}".format(epoch),
                    "train_loss= {:.4f}".format(loss_train_val),
                    "train_acc= {:.4f}".format(train_acc_sel),
                    "val_loss= {:.4f}".format(loss_valid_val),
                    "val_acc= {:.4f}".format(valid_acc_sel),
                    "test_loss= {:.4f}".format(loss_test_val),
                    "test_acc= {:.4f}".format(test_acc_sel),
                    "F1-score={:.4f}".format(f1),
                    "time= {:.4f}".format(time.time() - t))
                
                #最好的结果仍旧是准确率最高的结果
                if (test_acc_sel > best_acc):
                    best_acc = test_acc_sel
                    best_loss = loss_test_val
                    best_result = (epoch, loss_train_val, loss_valid_val, loss_test_val, 
                        result_table)
                    best_yuce_tag=preds_test
                    torch.save(self.model.state_dict(), '/home/kayzhou/zhangyue/text/text_UNION/test_GCN_LSTM/saved_models/%s' % self.MODEL_NAME)
                #用于判断是否结束训练

            else:
                print("Epoch: {:04d}".format(epoch),
                    "time= {:.4f}".format(time.time() - t))
                
            

            if loss_train< best_loss_train:
              best_loss_train = loss_train
              best_epoch = epoch

            if epoch-best_epoch > 10:
              print("Too long time not up")
              break
          

        print("best_result:")
        print(best_result)
            
            
            
            

        #print(best_result)
        fout = open('/home/kayzhou/zhangyue/text/text_UNION/test_GCN_LSTM/logs/result_%s.txt' % self.DATASET, 'a')
        fout.write(str(best_result) + '\n')
        fout.close()
        return best_yuce_tag,labels_test,loss_list, train_acc_list, dev_acc_list, test_acc_list




    def _evaluate(self, verbose=True):
        
        # Predict on full dataset
        embeds_0 = self.inputs[0] # model.input_layer()
        embeds_1 = self.model.gc1([embeds_0] + self.inputs[1:])
        embeds_2 = self.model.gc2([embeds_1] + self.inputs[1:])
        scores = self.model.clf_bias(embeds_2)
        preds = torch.argmax(scores, dim=1)
        loss_train = self.cross_entropy_loss(scores[self.idx_train_pre], self.labels_train)
        loss_valid = self.cross_entropy_loss(scores[self.idx_valid_pre], self.labels_valid)
        loss_test = self.cross_entropy_loss(scores[self.idx_test_pre], self.labels_test)
        correct_train = torch.sum(preds[self.idx_train_pre] == self.labels_train)
        correct_valid = torch.sum(preds[self.idx_valid_pre] == self.labels_valid)
        correct_test = torch.sum(preds[self.idx_test_pre] == self.labels_test)
        
        #计算准确率
        train_acc_net = correct_train.item()/self.labels_train.size(0)
        valid_acc_net = correct_valid.item()/self.labels_valid.size(0)
        test_acc_net = correct_test.item()/self.labels_test.size(0)
        
        if verbose:
            #整张图上的准确率
            print('Graph:', round(train_acc_net,4), round(valid_acc_net,4),round(test_acc_net,4))


        scores_shareu = []
        #scores_value = scores.data.numpy()
        scores_value = scores.data.cpu().numpy()
        for node in range(self.num_nodes):
            scores_t = np.zeros(2) 
            
            #self.node2adj为doc节点 to share用户节点
            if (node not in self.node2adj):
                scores_shareu.append(scores_t)
                continue
            
            #分享文档的用户集合
            adj = list(self.node2adj[node])
            
            adj_coef = [1/len(adj)] * len(adj)
            
            #根据加权分享该文档的用户节点的预测得到文档预测
            for user, coef in zip(adj, adj_coef):
                scores_t += coef * scores_value[user] 
            scores_shareu.append(scores_t) 
            
        #scores_shareu = torch.FloatTensor(np.array(scores_shareu))
        scores_shareu = torch.FloatTensor(np.array(scores_shareu)).cuda()
        preds_shareu = torch.argmax(scores_shareu, dim=1)
        correct_train_shareu = torch.sum(preds_shareu[self.idx_train_pre] == self.labels_train)
        correct_valid_shareu = torch.sum(preds_shareu[self.idx_valid_pre] == self.labels_valid)
        correct_test_shareu = torch.sum(preds_shareu[self.idx_test_pre] == self.labels_test)
        train_acc_shareu = correct_train_shareu.item()/self.labels_train.size(0)
        valid_acc_shareu = correct_valid_shareu.item()/self.labels_valid.size(0)
        test_acc_shareu = correct_test_shareu.item()/self.labels_test.size(0)
        if verbose:
            print('User:', round(train_acc_shareu,4), round(valid_acc_shareu,4), round(test_acc_shareu,4))

        #结合图预测的文档分类和用户预测的文档节点分类
        scores_shareu += scores
        preds_netshareu = torch.argmax(scores_shareu, dim=1)
        correct_train_shareu = torch.sum(preds_netshareu[self.idx_train_pre] == self.labels_train)
        correct_valid_shareu = torch.sum(preds_netshareu[self.idx_valid_pre] == self.labels_valid)
        correct_test_shareu = torch.sum(preds_netshareu[self.idx_test_pre] == self.labels_test)
        train_acc_netshareu = correct_train_shareu.item()/self.labels_train.size(0)
        valid_acc_netshareu = correct_valid_shareu.item()/self.labels_valid.size(0)
        test_acc_netshareu = correct_test_shareu.item()/self.labels_test.size(0)
        if verbose:
            print('G+U:', round(train_acc_netshareu,4), round(valid_acc_netshareu,4), round(test_acc_netshareu,4))
        
        #-----------------
        text_idx_perm = [i for i in range(self.num_docs)]
        scores_text = []
        for start in range(0,self. num_docs, self.text_batch_size):
            self.model.zero_grad()
            end = start + self.text_batch_size
            if (end > self.num_docs):
                end = self.num_docs
            doc_idx_list_raw = text_idx_perm[start:end]
            #doctext_idx_list = torch.LongTensor(doc_idx_list_raw)
            doctext_idx_list = torch.LongTensor(doc_idx_list_raw).cuda()
            batch_input = self.model.input_layer.get_doc_embed(doctext_idx_list)
                # torch.mm(model.gc2.W[0])
            scores_text.extend(list(self.model.clf_bias(batch_input).data.cpu().numpy()))
            #scores_text.extend(list(self.model.clf_bias(batch_input).data.numpy()))
            #！！！！！！！！！！！！！！！！！！！！！！！修改
            #scores_text.extend(list(batch_input).cpu().numpy())
        #scores_text = torch.FloatTensor(scores_text)
        
        scores_text = np.array(scores_text)
        scores_text = torch.FloatTensor(scores_text).cuda()
        #scores_text = torch.FloatTensor(scores_text)
        preds_text = torch.argmax(scores_text, dim=1)
        # print(idx_train-num_non_docs)
        # print(preds_shareu[idx_train-num_non_docs])
        # exit()
        correct_train = torch.sum(preds_text[self.idx_train] == self.labels_train)
        correct_valid = torch.sum(preds_text[self.idx_valid] == self.labels_valid)
        correct_test = torch.sum(preds_text[self.idx_test] == self.labels_test)
        train_acc_text = correct_train.item()/self.labels_train.size(0)
        valid_acc_text = correct_valid.item()/self.labels_valid.size(0)
        test_acc_text = correct_test.item()/self.labels_test.size(0)

        loss_text_train = self.cross_entropy_loss(scores_text[self.idx_train], self.labels_train)
        loss_text_valid = self.cross_entropy_loss(scores_text[self.idx_valid], self.labels_valid)
        loss_text_test = self.cross_entropy_loss(scores_text[self.idx_test], self.labels_test)

        if verbose:
            print('Text:', round(train_acc_text,4), round(valid_acc_text,4), round(test_acc_text,4))



        
        #scores_g_t = torch.cat((scores[self.num_non_docs:],scores_text),dim=1)
        scores[self.num_non_docs:] += scores_text
        preds_nettext = torch.argmax(scores, dim=1)
        correct_train = torch.sum(preds_nettext[self.idx_train_pre] == self.labels_train)
        correct_valid = torch.sum(preds_nettext[self.idx_valid_pre] == self.labels_valid)
        correct_test = torch.sum(preds_nettext[self.idx_test_pre] == self.labels_test)
        train_acc_nettext = correct_train.item()/self.labels_train.size(0)
        valid_acc_nettext = correct_valid.item()/self.labels_valid.size(0)
        test_acc_nettext = correct_test.item()/self.labels_test.size(0)
        if verbose:
            print('G+T:', round(train_acc_nettext,4), round(valid_acc_nettext,4), round(test_acc_nettext,4))

        
        #scores_shareu[self.num_non_docs:] = torch.cat((scores_shareu[self.num_non_docs:],scores_text),dim=1)
        scores_shareu[self.num_non_docs:] += scores_text
        preds_all = torch.argmax(scores_shareu, dim=1)
        correct_train = torch.sum(preds_all[self.idx_train_pre] == self.labels_train)
        correct_valid = torch.sum(preds_all[self.idx_valid_pre] == self.labels_valid)
        correct_test = torch.sum(preds_all[self.idx_test_pre] == self.labels_test)
        train_acc_all = correct_train.item()/self.labels_train.size(0)
        valid_acc_all = correct_valid.item()/self.labels_valid.size(0)
        test_acc_all = correct_test.item()/self.labels_test.size(0)

        loss_all_train = self.cross_entropy_loss(scores_shareu[self.idx_train_pre], self.labels_train)
        loss_all_valid = self.cross_entropy_loss(scores_shareu[self.idx_valid_pre], self.labels_valid)
        loss_all_test = self.cross_entropy_loss(scores_shareu[self.idx_test_pre], self.labels_test)
        

        if verbose:
            print('G+U+T:', round(train_acc_all,4), round(valid_acc_all,4), round(test_acc_all,4))



        
        if (self.PRED_TYPE == 'net'):
            loss_train,loss_valid,loss_test,train_acc_sel, valid_acc_sel, test_acc_sel, preds_sel = (
                loss_train.item(),loss_valid.item(),loss_test.item(),
                train_acc_net, valid_acc_net, test_acc_net, preds.data.cpu().numpy())
        elif (self.PRED_TYPE == 'netshareu'):
            loss_train,loss_valid,loss_test,train_acc_sel, valid_acc_sel, test_acc_sel, preds_sel = (
                loss_train.item(),loss_valid.item(),loss_test.item(),
                train_acc_netshareu, valid_acc_netshareu, test_acc_netshareu, preds_netshareu.data.cpu().numpy())
        elif (self.PRED_TYPE == 'text'):
            loss_train,loss_valid,loss_test,train_acc_sel, valid_acc_sel, test_acc_sel, preds_sel = (
                loss_text_train.item(),loss_text_valid.item(),loss_text_test.item(),
                train_acc_text, valid_acc_text, test_acc_text, preds_text.data.cpu().numpy())
        elif (self.PRED_TYPE == 'all'):
            loss_train,loss_valid,loss_test,train_acc_sel, valid_acc_sel, test_acc_sel, preds_sel = (
                loss_all_train.item(),loss_all_valid.item(),loss_all_test.item(),
                train_acc_all, valid_acc_all, test_acc_all, preds_all.data.cpu().numpy())
        else:
            print('wrong PRED_TYPE')
            exit()
        
        result_table = [[train_acc_net, valid_acc_net, test_acc_net], 
            [train_acc_shareu, valid_acc_shareu, test_acc_shareu], 
            [train_acc_netshareu, valid_acc_netshareu, test_acc_netshareu], 
            [train_acc_text, valid_acc_text, test_acc_text], 
            [train_acc_nettext, valid_acc_nettext, test_acc_nettext], 
            [train_acc_all, valid_acc_all, test_acc_all]]
        # return loss_list, train_acc_list, dev_acc_list, test_acc_list
        return (preds_all[self.idx_test_pre],self.labels_test,loss_train,loss_valid,loss_test,train_acc_sel,
                valid_acc_sel, test_acc_sel, preds_sel, result_table)
 

        
    
    def run(self):
            
            # Train model
            best_yuce_tag,labels_test,loss_list, train_acc_list, dev_acc_list, test_acc_list = self._train()
        
            # Evaluate trained model
            self.model.load_state_dict(torch.load('/home/kayzhou/zhangyue/text/text_UNION/test_GCN_LSTM/saved_models/%s' % (self.MODEL_NAME)), strict=False)
            # (loss_train_val, loss_valid_val, loss_test_val, 
            #     train_acc_sel, valid_acc_sel, test_acc_sel, preds_sel, result_table) = self._evaluate()
            # print(np.array(result_table)[:, 2])

            if (self.SUPERVISE_FLAG == 'unsup1'):
                fout = open('temp/%s_unsup_pred.pickle' % self.DATASET, 'wb')
                preds_numpy = preds_sel
                print(preds_numpy.shape)
                pkl.dump(preds_numpy, fout)
                fout.close()
            
            return best_yuce_tag,labels_test,loss_list, train_acc_list, dev_acc_list, test_acc_list


if __name__ == '__main__':
    # Hyper Parameters
    ap = argparse.ArgumentParser()
    ap.add_argument("-d", "--dataset", type=str, default="one_month_network_202010",
                    help="Dataset string")
    ap.add_argument("-e", "--epochs", type=int, default=1000,
                    help="Number training epochs")
    ap.add_argument("-hd", "--hidden", type=int, default=16,
                    help="Number hidden units")
    ap.add_argument("-b", "--bases", type=int, default=-1,
                    help="Number of bases used (-1: all)")
    ap.add_argument("-lr", "--learnrate", type=float, default=0.001,
                    help="Learning rate")
    ap.add_argument("-l2", "--l2norm", type=float, default=5e-4,
                    help="L2 normalization of input weights")
    ap.add_argument("-tm", "--text_model", type=str, default="HLSTM",
                    help="text model: select from 'SkipThought'/'HLSTM'")
    ap.add_argument("-sf", "--supervise_flag", type=str, default="supervise",
                    help="supervise flag: select from 'supervise'/'unsup1'/'unsup2'")
    ap.add_argument("-pt", "--pred_type", type=str, default="all",
                    help="prediction method: select from 'net'/'netshareu'/'text'/No doc'")
    args = vars(ap.parse_args(args=[]))
    dataset = args['dataset']

    
    np.random.seed(1)
    torch.manual_seed(1)
    torch.cuda.manual_seed(1)
    

    trainer = Trainer(args)
    best_yuce_tag,labels_test,loss_list, train_acc_list, dev_acc_list, test_acc_list = trainer.run()  



{'dataset': 'one_month_network_202010', 'epochs': 1000, 'hidden': 16, 'bases': -1, 'learnrate': 0.001, 'l2norm': 0.0005, 'text_model': 'HLSTM', 'supervise_flag': 'supervise', 'pred_type': 'all'}
898999 112376 112374
y:1123749
num_nodes:1435367
support:1
2
Data loaded successfully!


1123750it [00:08, 124866.42it/s]
100%|██████████| 1193517/1193517 [01:14<00:00, 16099.42it/s]


1123749 64 826239 1123749
1--loss_train
tensor(1614.2800, device='cuda:0', grad_fn=<AddBackward0>)
Graph: 0.6444 0.6433 0.6447
User: 0.6444 0.6433 0.6447
G+U: 0.6444 0.6433 0.6447
Text: 0.6444 0.6433 0.6447
G+T: 0.6444 0.6433 0.6447
G+U+T: 0.6444 0.6433 0.6447
Epoch: 0001 train_loss= 644460.2500 train_acc= 0.6444 val_loss= 80798.4531 val_acc= 0.6433 test_loss= 80566.6875 test_acc= 0.6447 F1-score=0.3920 time= 1988.2489
2--loss_train
tensor(1620.3666, device='cuda:0', grad_fn=<AddBackward0>)
Graph: 0.6444 0.6433 0.6447
User: 0.6444 0.6433 0.6447
G+U: 0.6444 0.6433 0.6447
Text: 0.6444 0.6433 0.6447
G+T: 0.6444 0.6433 0.6447
G+U+T: 0.6444 0.6433 0.6447
Epoch: 0002 train_loss= 639249.2500 train_acc= 0.6444 val_loss= 80702.0391 val_acc= 0.6433 test_loss= 80449.8984 test_acc= 0.6447 F1-score=0.3920 time= 1982.0639
3--loss_train
tensor(1607.6378, device='cuda:0', grad_fn=<AddBackward0>)
Graph: 0.6444 0.6433 0.6447
User: 0.6444 0.6433 0.6447
G+U: 0.6444 0.6433 0.6447
Text: 0.6444 0.6433 0.6447